# Практическая работа №7

# Выполнил студент группы ББМО-01-23 Курдюков А.А.

## Шаги выполнения

### Шаг 1: Защита с помощью Adversarial Training




In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist

# Загрузка данных MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Нормализация данных
train_images = train_images / 255.0
test_images = test_images / 255.0

# Преобразование меток в one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

# Уменьшаем количество обучающих изображений до 1000
train_images = train_images[:1000]
train_labels = train_labels[:1000]

# Функция FGSM атаки
def fgsm_attack(image, epsilon, gradient):
    perturbation = epsilon * np.sign(gradient)
    adversarial_image = image + perturbation
    adversarial_image = np.clip(adversarial_image, 0, 1)  # Ограничение значений пикселей
    return adversarial_image

# Функция для генерации противоречивых примеров
def generate_adversarial_examples(model, images, labels, epsilon):
    adversarial_images = []
    for i in range(len(images)):
        image = tf.convert_to_tensor(images[i].reshape((1, 28, 28, 1)), dtype=tf.float32)  # Преобразование в tf.Tensor
        label = tf.convert_to_tensor(labels[i].reshape((1, 10)), dtype=tf.float32)  # Преобразование метки в Tensor

        with tf.GradientTape() as tape:
            tape.watch(image)
            prediction = model(image)
            loss = tf.keras.losses.categorical_crossentropy(label, prediction)
        gradient = tape.gradient(loss, image)
        adversarial_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())  # Конвертируем обратно в numpy
        adversarial_images.append(adversarial_image.reshape(28, 28))  # Преобразуем в форму (28, 28)

    return np.array(adversarial_images)

# Создание модели
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Обучение модели с противоречивыми примерами
def adversarial_training(model, train_images, train_labels, epsilon):
    for epoch in range(5):  # Короткое обучение для демонстрации
        for i in range(0, len(train_images), 32):
            batch_images = train_images[i:i+32]
            batch_labels = train_labels[i:i+32]
            # Генерация противоречивых примеров для текущей партии данных
            adversarial_images = generate_adversarial_examples(model, batch_images, batch_labels, epsilon)
            # Объединение оригинальных и противоречивых примеров
            combined_images = np.concatenate([batch_images, adversarial_images], axis=0)
            combined_labels = np.concatenate([batch_labels, batch_labels], axis=0)
            # Обучение на комбинированных данных
            model.train_on_batch(combined_images, combined_labels)

# Инициализация модели
model = create_model()

# Тренировка модели с защитой на противоречивых примерах
adversarial_training(model, train_images, train_labels, epsilon=0.1)

# Сохранение защищенной модели
model.save('adversarial_trained_model.h5')

###Шаг 2: Градиентная маскировка (Gradient Masking)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Activation
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Загрузка и подготовка данных
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Нормализация данных
train_images = train_images / 255.0
test_images = test_images / 255.0

# Преобразование меток в one-hot encoding
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

# Обновление модели для градиентной маскировки
def create_masked_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10),  # Без активации на последнем слое
        Activation('softplus')  # Используем softplus вместо softmax для градиентной маскировки
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Обучение модели с градиентной маскировкой
masked_model = create_masked_model()
masked_model.fit(train_images, train_labels, epochs=5)

# Сохранение модели с градиентной маскировкой
masked_model.save('masked_model.h5')

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.8795 - loss: 0.4511
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9638 - loss: 0.1247
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9753 - loss: 0.0842
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9804 - loss: 0.0641
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9850 - loss: 0.0491


## Шаг 3: Регуляризация и нормализация для повышения устойчивости

In [ ]:
# Модель с регуляризацией и нормализацией
def create_regularized_model():
 model = tf.keras.Sequential([
 tf.keras.layers.Flatten(input_shape=(28, 28)),
 tf.keras.layers.Dense(128, activation='relu',
kernel_regularizer=tf.keras.regularizers.l2(0.01)),
 tf.keras.layers.Dropout(0.5),tf.keras.layers.BatchNormalization(),
 tf.keras.layers.Dense(10, activation='softmax')
 ])
 model.compile(optimizer='adam', loss='categorical_crossentropy',
metrics=['accuracy'])
 return model
# Обучение модели с регуляризацией и нормализацией
regularized_model = create_regularized_model()
regularized_model.fit(train_images, train_labels, epochs=5)
# Сохранение модели с регуляризацией
regularized_model.save('regularized_model.h5')

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7944 - loss: 1.3445
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8670 - loss: 0.6272
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8644 - loss: 0.6183
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8584 - loss: 0.6296
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8519 - loss: 0.6299


###Шаг 4:  Оценка моделей на противоречивых примерах

In [ ]:
# Загрузка атакованной модели
protected_model = tf.keras.models.load_model('adversarial_trained_model.h5')
# Генерация противоречивых примеров для тестовых данных
adversarial_test_images = generate_adversarial_examples(protected_model,
test_images, test_labels, epsilon=0.1)
# Оценка защищенной модели на противоречивых примерах
test_loss, test_acc = protected_model.evaluate(adversarial_test_images,test_labels)
print(f'Accuracy of protected model on adversarial examples: {test_acc}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4909 - loss: 1.4584
Accuracy of protected model on adversarial examples: 0.5533999800682068


In [11]:
# Загрузка атакованной модели
regularized_model = tf.keras.models.load_model('regularized_model.h5')
masked_model = tf.keras.models.load_model('masked_model.h5')

# Оценка модели с регуляризацией и нормализацией на противоречивых примерах
test_loss, test_acc = regularized_model.evaluate(adversarial_test_images,test_labels)
print(f'Accuracy of regularized model on adversarial examples: {test_acc}')

# Оценка модели с градиентной маскировкой на противоречивых примерах
test_loss, test_acc = masked_model.evaluate(adversarial_test_images,test_labels)
print(f'Accuracy of masked model on adversarial examples: {test_acc}')


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7163 - loss: 1.0090
Accuracy of regularized model on adversarial examples: 0.7599999904632568
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6308 - loss: 1.4771
Accuracy of masked model on adversarial examples: 0.6704000234603882


#Вывод:
В ходе работы была проведена реализация защиты модели от атак с использованием противоречивых примеров (adversarial examples). Для этого использовались методы генерации противоречивых примеров с помощью атаки FGSM (Fast Gradient Sign Method), градиентная маскировка (masked model) и модель с регуляризацией и нормализацией. Модель была обучена на данных, включающих как оригинальные изображения, так и специально созданные противоречивые примеры, что должно было улучшить её устойчивость к атакам. Для реализации градиентной маскировки в модели использовалась функция активации softplus на последнем слое вместо стандартного softmax, что позволило смягчить влияние резких градиентов и повысить устойчивость к атакам.

Результаты оценки защищенных моделей на противоречивых примерах показали следующие результаты. Модель с градиентной маскировкой продемонстрировала точность 0.67, что является хорошим результатом, однако она уступала модели с регуляризацией и нормализацией, которая показала значительно более высокую точность — 0.7599. Эти результаты свидетельствуют о том, что защита с регуляризацией и нормализацией на противоречивых примерах оказалась наиболее эффективной из всех исследованных методов. Точность модели с регуляризацией и нормализацией на противоречивых примерах составила 75,99%, что на 8,29% выше, чем у модели с градиентной маскировкой, что указывает на её большую способность устойчивости к атакам.

Таким образом, можно сделать вывод, что среди рассмотренных методов защиты наиболее эффективной оказалась модель с регуляризацией и нормализацией. Этот метод показал наибольшую точность на противоречивых примерах, что подтверждает его высокую эффективность в защите от атак. Модель с градиентной маскировкой также показала хорошие результаты, но уступила в производительности модели с регуляризацией.